In [ ]:
import os
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import math
import calendar

In [ ]:
os.chdir("/Users/giandomenico/Documents/SAPIENZA/Papers/PBL_Failure Hazard/data")


In [ ]:
data = gpd.read_file("crolli_2022_3857_membro.shp").set_index('id')
data['Data'] = pd.to_datetime(data['Data'])
data = data.sort_values(by='Data')
data['Litologia'] = data['Litologia'].replace({'Marna': 'Marl','marna': 'Marl', 'Arenaria':'Sandstone', 'arenaria':'Sandstone'})

# Select from May 2022 to May 2023
data = data.set_index(['Data'])
data = data.loc['2022-05-01': '2023-05-31']
time_interval = (data.index.max() - data.index.min()).days+1
areas = data['area_m'].values
times = data.index.values

In [ ]:
area_pml = 14356
area_pml_marl = round(area_pml*0.3, 1)
area_pml_sandstone = round(area_pml*0.7, 1)

area_pa = 15444
area_pa_marl = round(area_pa*0.5, 1)
area_pa_sandstone = round(area_pa*0.5, 1)

In [ ]:
#%% Descriptive Statistics on Rockfall events.
stat = data.describe()
stat_lito = data.groupby('Litologia')['area_m'].describe()

In [ ]:
# HISTOGRAMS
# create unique value for each group
rf_mag_giu_lug22['group'] = 1
rf_ago_set_ott22['group'] = 2
rf_nov_dic_gen2223['group'] = 3
rf_fb_mar_apr_mag23['group'] = 4
# concat the groups
rf_groups_months = pd.concat([rf_mag_giu_lug22, rf_ago_set_ott22, rf_nov_dic_gen2223, rf_fb_mar_apr_mag23], ignore_index=True)
# rf_groups_months.to_file("crolli_grouped_months.gpkg", driver='GPKG')

#%% plot hist stacked by groups
# Define the color palette
colors = ['#648FFF', '#DC267F', '#FE6100', '#FFB000']
# Define the legend labels
legend_labels = ['Feb\'23-May\'23', 'Nov\'22-Jan\'23','Aug\'22-Oct\'22', 'May\'22-July\'22']

plt.figure(figsize=(7,5), dpi=600, facecolor='w', edgecolor='black')
sns.histplot(data=rf_groups_months, x='area_m', hue='group', stat='count',
             shrink=.8, multiple='stack', element='bars', bins=15, binwidth=.5, fill=1,
             log_scale=(True, False), cumulative=0, kde=0,
               palette=colors, zorder=2
             )

# inverting the x-axis
# plt.gca().invert_xaxis()
# plt.gca().invert_yaxis()

plt.xlabel('Area ($\mathrm{m^{2}}$)', fontsize=14)
plt.ylabel('Rockfalls Count', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

lw = 1.4
plt.gca().spines['top'].set_linewidth(lw)
plt.gca().spines['bottom'].set_linewidth(lw)
plt.gca().spines['left'].set_linewidth(lw)
plt.gca().spines['right'].set_linewidth(lw)
plt.grid(axis='y', color='gray', alpha=0.15, zorder=1, lw=1)

# Change the legend labels
plt.legend(labels=legend_labels, fontsize=14, frameon=0)

# plt.savefig("figures/rf_Hist_temporal_occurrence.png", dpi=600, bbox_inches='tight')
plt.show()

## Cumulative Frequency-Magnitude Curves

In [ ]:

def hazard_curve(data, col, time_interval, N=33, title='Frequency - Magnitude Relations'):
    V_max = data[col].max()+1
    sorted_vol = np.sort(data[col].values)[::-1]
    cumulative_frequencies = np.cumsum(sorted_vol < V_max)
    total_area_hm2 = 29800 / 10000
    total_time_years = time_interval /365
    normalized_cumulative_frequencies = cumulative_frequencies / (total_area_hm2 * total_time_years)
    
    # Slice to fit power law
    V_SEL = sorted_vol[:-N]
    freq_SEL = normalized_cumulative_frequencies[:-N]
    def power_law(x, a, b):
        return a * np.power(x, b)
    
    # Fit the power-law curve
    params, covariance = curve_fit(power_law, V_SEL, freq_SEL)
    a, b = params
    print(f'Fitted Parameters: a = {a:.2f}, b = {b:.2f}')
    # Display the equation of the fitted curve
    equation = f'λ$_{{st}}$ = {a:.2f} * A^{b:.2f}'
    print(f'Equation: {equation}')
    # Compute R-squared value
    predicted_cumulative_frequencies = power_law(V_SEL, a, b)
    r_squared = r2_score(freq_SEL, predicted_cumulative_frequencies)
    print(f'R-squared: {r_squared}')
    # OVERALL FAILURE FREQUENCY
    failure_frequency = len(data) / time_interval
    # n = 1
    # Pf = ((failure_frequency**n) / np.math.factorial(n) ) * math.exp(-1*failure_frequency)
    Pf = 1 - math.exp(-1*failure_frequency)
    print(f"The failure probability is: {Pf:.2f}")
    
    # Plot the Fitted Curve
    plt.figure(figsize=(7,5), dpi=600)
    plt.scatter(sorted_vol, normalized_cumulative_frequencies, marker='o', color='k', alpha=0.5, label='Rockfall Events')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Area ($\mathrm{m^{2}}$)', fontsize=11)
    plt.ylabel('Normalised Cumulative Frequency \n($\mathrm{hm^{-2} \cdot year^{-1}}$)', fontsize=11)
    plt.title(title, fontsize=13)

    x_fit = np.logspace(np.log10(min(V_SEL)), np.log10(max(V_SEL)), 100)
    y_fit = power_law(x_fit, a, b)
    plt.plot(x_fit, y_fit, color='r', label='Power-Law Fit', lw=2, ls='dashed')
    

    # Annotate the plot with the equation and R-squared value
    equation_text = f'{equation}\nR-squared: {r_squared:.2f}\nPf: {Pf:.2f}'
    plt.annotate(equation_text, xy=(0.1, 0.1), xycoords='axes fraction', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))

    plt.grid(True, which="both", ls="--", lw=0.4)

    plt.legend(fontsize=11)
    # plt.savefig(f'figures/{title}.png', dpi=600)
    # plt.savefig(f'figures/{title}.svg', format='SVG')
    plt.show()
            
    return

In [ ]:
#%% COMPARE HAZARD CURVES
def compare_hazard_curve(data, col, time_interval, by='Membro', total=False):
    # create list with sectors
    sectors = np.unique(data[by]).tolist()
    # sectors.append('Total')
    total_time_years = time_interval / 365  # Convert from days to years
    # Set N
    N = 25
    plt.figure(figsize=(7,5), dpi=600)
    # iterate over the sectors
    for s in sectors:
        # crete subset of data
        if total:
            df = data
        else:
            df = data[data[by] == s]
        # By Lithology
        if total:
            color, fit_color, label, xy = 'k', 'r','Rockfall Events', (0.25, 0.15)
            # Define the total area
            total_area_hm2 = 29800 / 10000  # Convert from square meters to hectometers squared
        elif s == 'Marl':
            color, fit_color, label, xy = '#0C7BDC', '#0C7BDC','Rockfalls in Marls', (0.25,0.15)
            total_area_hm2 = 12028.8 / 10000
        elif s == 'Sandstone':
            color, fit_color, label, xy = '#FFC20A', '#FFC20A','Rockfalls in Sandstone', (0.65,0.5)
            total_area_hm2 = 17771.2 / 10000
        # By Formation
        elif s == 'pa':
            color, fit_color, label, xy = '#0C7BDC', '#0C7BDC','Rockfalls from PA', (0.25,0.15)
            total_area_hm2 = 15444 / 10000
        elif s == 'pml':
            color, fit_color, label, xy = '#FFC20A', '#FFC20A','Rockfalls from PML', (0.65,0.5)
            total_area_hm2 = 14356 / 10000
        else: pass
        # Compute volumes and frequencies
        V_max = df[col].max()+1
        sorted_vol = np.sort(df[col].values)[::-1]
        cumulative_frequencies = np.cumsum(sorted_vol < V_max)
        normalized_cumulative_frequencies = cumulative_frequencies / (total_area_hm2 * total_time_years)
        # Slice to fit power law
        V_SEL = sorted_vol[:-N]
        freq_SEL = normalized_cumulative_frequencies[:-N]
        def power_law(x, a, b):
            return a * (x ** b)
        # Fit the power-law curve
        params, covariance = curve_fit(power_law, V_SEL, freq_SEL)
        a, b = params
        print(f'Fitted Parameters for {s}: a = {a}, b = {b}')
        # Display the equation of the fitted curve
        equation = f'λ$_{{st}}$ = {a:.2f} * A^{b:.2f}'
        print(f'Equation for {s}: {equation}')
        # Compute R-squared value
        predicted_cumulative_frequencies = power_law(V_SEL, a, b)
        r_squared = r2_score(freq_SEL, predicted_cumulative_frequencies)
        print(f'R-squared for {s}: {r_squared}')
        # OVERALL FAILURE FREQUENCY
        failure_frequency = len(df) / time_interval
        # n = 1
        # Pf = ((failure_frequency**n) / np.math.factorial(n) ) * math.exp(-1*failure_frequency)
        Pf = 1 - math.exp(-1*failure_frequency)
        print(f"The failure probability of {s} is: {Pf:.2f}")
    
        # Plot the Fitted Curve
        
        plt.scatter(sorted_vol, normalized_cumulative_frequencies, marker='o', color=color, edgecolor='k', alpha=0.5, label=label)
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('Area ($\mathrm{m^{2}}$)', fontsize=11)
        plt.ylabel('Normalised Cumulative Frequency\n($\mathrm{hm^{-2} \cdot year^{-1}}$)', fontsize=11)
        # plt.title('Frequency - Magnitude Relations', fontsize=13)
    
        x_fit = np.logspace(np.log10(min(V_SEL)), np.log10(max(V_SEL)), 50)
        y_fit = power_law(x_fit, a, b)
        plt.plot(x_fit, y_fit, color=fit_color, label='Power-Law Fit', lw=3, ls='dashed')
        
        # Annotate the plot with the equation and R-squared value
        equation_text = f'{equation}\nR-squared: {r_squared:.2f}\nP$_{{f}}$: {Pf:.2f}'
        plt.annotate(equation_text, xy=xy, xycoords='axes fraction', fontsize=10.5, 
                     bbox=dict(boxstyle="round,pad=0.3", edgecolor=fit_color, facecolor='white', lw=.5))
    
    plt.grid(which='both', axis='both', lw=0.2)
    plt.legend(fontsize=11)
    # plt.savefig(f'figures/H_Curve_by_{by}.png')
    # plt.savefig(f'figures/H_Curve_by_{by}.svg', format='SVG', bbox_inches='tight')
    plt.show()
    
    return

In [ ]:
hazard_curve(data, col='area_m', time_interval=time_interval, N=33)

In [ ]:
compare_hazard_curve(data, 'area_m', time_interval, by='Membro')

In [ ]:
compare_hazard_curve(data, 'area_m', time_interval, by='Litologia')